## Importando bibliotecas necessárias

In [2]:
import pandas as pd
from datetime import datetime

### Importação da base de dados e tratamento incial
- Substituição de células vazias por 'presente', de acordo com o recomendado pelo provedor da base de dados

In [33]:
df = pd.read_csv('C:\\Users\\josen\\Downloads\\DatalinkBI\\Trabalhos\\ateliedadiele\\fluxoclientesateliedadiele.csv')
df['presenca'] = df['presenca'].fillna('presente')
df.iat[0, 2] = '05/01/2023'
df.iat[754, 2] = '29/09/2023'
df.iat[761, 2] = '04/10/2023'
df.iat[1000, 2] = '28/12/2023'

### Padronizando formas de escrita do mesmo nome
- O nome do mesmo cliente pode estar escrito de formas diferentes, abaixo se encontra a listagem das possibilidade

In [5]:
#Os dados dessa seção foram censurados devido a questões de privacidade com nomes dos clientes da empresa

# Criando conjunto csv da análise de cada cliente
- Cada cliente possui um conjunto de arquivos csv que posteriormente serão utilizados para realizar análises dentro do software Power BI

#### Definindo horários, serviços e dias da semana mais procurados por cada cliente
- Foi utilizada uma lógica de iteração sob indices;
- Cada cliente possui um índicie, como indicado na lista 'nomes_clientes'
- A mesma lógica de indicie foi aplicada para a contagem e definição das variáveis únicas

In [6]:
#===Definindo matrizes de horários, serviços e dias da semana com índicies equivalentes a da lista 'nomes_clientes'
servicos_procurados = [[] for _ in range(35)]
horarios_procurados = [[] for _ in range(35)]
diasdasemana_procurados = [[] for _ in range(35)]

dias_da_semana = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']

for i in range(len(df)):
    servico = df['servicos'].iloc[i].split()
    cliente = df['cliente'].iloc[i]
    horario = df['horario'].iloc[i]
    data = df['data'].iloc[i]
    
    dia_da_semana = datetime.strptime(data, '%d/%m/%Y').weekday()
    
    index = 0
    for possibilidade in possibilidades_clientes:
        if cliente in possibilidade:
            for i in servico:
                servicos_procurados[index].append(i)
            horarios_procurados[index].append(horario)
            diasdasemana_procurados[index].append(dias_da_semana[dia_da_semana])
            break
        
        index += 1

#===Quantidade por dia da semana procurado
dataframes_diasemana = []
    
for diassemana in diasdasemana_procurados:
    quantidades = []
    
    for diasemana in dias_da_semana:
        quantidade = diassemana.count(diasemana)
        quantidades.append(quantidade)
        
    df_diasemana = pd.DataFrame({'dia-da-semana' : dias_da_semana, 'quantidade' : quantidades})
    dataframes_diasemana.append(df_diasemana)
    
#===Quantidade por serviço procurado
dataframes_servicos = []

for servicos in servicos_procurados:
    servicos_unicos = list(set(servicos))
    quantidades = []
    
    for servico in servicos_unicos:
        quantidade = servicos.count(servico)
        quantidades.append(quantidade)
    
    df_servicos = pd.DataFrame({'servico' : servicos_unicos, 'quantidade' : quantidades})
    df_servicos = df_servicos.sort_values(by='quantidade')
    dataframes_servicos.append(df_servicos)

#===Quantidade por horário procurado
dataframes_horarios = []

for horarios in horarios_procurados:
    horarios_unicos = list(set(horarios))
    quantidades = []
    
    for horario in horarios_unicos:
        quantidade = horarios.count(horario)
        quantidades.append(quantidade)
    
    df_horarios = pd.DataFrame({'horario' : horarios_unicos, 'quantidade' : quantidades})
    df_horarios = df_horarios.sort_values(by='horario')
    dataframes_horarios.append(df_horarios)

#===Criação dos arquivos csv
for i in range(len(nomes_clientes)):
    pass
    dataframes_servicos[i].to_csv(f'C:\\Users\\\\downloads\\ateliedadiele\\perfilclientes\\servicos\\servicos-{nomes_clientes[i]}.csv', index=False)
    dataframes_horarios[i].to_csv(f'C:\\Users\\\\downloads\\ateliedadiele\\perfilclientes\\horarios\\horarios-{nomes_clientes[i]}.csv', index=False)
    dataframes_diasemana[i].to_csv(f'C:\\Users\\\\downloads\\ateliedadiele\\perfilclientes\\diasemana\\diasemana-{nomes_clientes[i]}.csv', index=False)

#### Definindo quanto em média é gasto por atendimento, total gasto por clientes, taxa de presença e datas
- Foi utilizada uma lógica de iteração sob indices;
- Cada cliente possui um índicie, como indicado na lista 'nomes_clientes'
- A mesma lógica de indicie foi aplicada para a contagem e definição das variáveis únicas

In [55]:
servicos_valores = {
    'pemao': 45,
    'mao': 25,
    'pe': 28,
    '': 0,
    'spa': 40,
    'pdp': 75,
    'axila': 20,
    'contorno': 45,
    'pmao': 10,
    'dc': 120,
    'buco': 15,
    'pemaao': 45,
    'ppe': 10,
    'sobrancelha': 25,
    'x': 25,
    'depilacao': 45,
    'desencravar': 28,
    'perna': 60,
    'blindagem': 80,
    'nariz': 10,
    'clareamentoax': 60,
    'claramentoax': 60,
    'clareamento': 60,
    'axilia': 20
}

#===Estruturando dados necessários para posteriores cálculos
quantidade_atendimentos = [[] for _ in range(35)]
valor_total = [[] for _ in range(35)]
ausencia_presenca = [[] for _ in range(35)]
datas_frequencia = [[] for _ in range(35)]

for i in range(len(df)):
    servico = df['servicos'].iloc[i].split()
    cliente = df['cliente'].iloc[i]
    situacao = df['presenca'].iloc[i]
    data = df['data'].iloc[i]
    datan = data[3:]
    
    index = 0
    for possibilidade in possibilidades_clientes:
        if cliente in possibilidade:
            total = 0
            for elemen in servico:
                valor = servicos_valores[elemen]
                total += valor
            valor_total[index].append(total)
            ausencia_presenca[index].append(situacao)
            datas_frequencia[index].append(datan)
            break
        index += 1

#===Calculando total de atendimentos por cliente
for i in range(len(valor_total)):
    quantidade_atendimentos[i].append(len(valor_total[i]))

#===Média por atendimento de cada cliente
media_por_atendimento = []
for i in range(len(valor_total)):
    media_por_atendimento.append(int(sum(valor_total[i]) / quantidade_atendimentos[i][0]))

dataframes_mediaatendimento = []
for i in range(len(nomes_clientes)):
    df_mediaatendimento = pd.DataFrame({'media':[media_por_atendimento[i]], 'cliente':[nomes_clientes[i]]})
    dataframes_mediaatendimento.append(df_mediaatendimento)
    
#===Calculando o total gasto por cada cliente
total_atendimentos = []
for valor in valor_total:
    total_atendimentos.append(sum(valor))

dataframes_totalatendimentos = []
for i in range(len(nomes_clientes)):
    df_totalatendimentos = pd.DataFrame({'total':[total_atendimentos[i]], 'cliente':[nomes_clientes[i]]})
    dataframes_totalatendimentos.append(df_totalatendimentos)
    
#===Calculando taxa de presença e ausencia
dataframes_presenca = []
for situacao in ausencia_presenca:
    taxa_presente = ((100 * situacao.count('presente')) / len(situacao))
    try:
        taxa_ausente = ((100 * situacao.count('ausente')) / len(situacao))
    except:
        taxa_ausente = 0
    df_presenca = pd.DataFrame({'presente':[round(taxa_presente,2)], 'ausente':[round(taxa_ausente,2)]})
    dataframes_presenca.append(df_presenca)
    
#===Data de marcação do atendimento
dataframes_data = []
datas_unicas = ['01/2023', '02/2023', '03/2023', '04/2023', '05/2023', '06/2023', '07/2023', '08/2023', '09/2023', '10/2023', '11/2023', '12/2023']
for datas in datas_frequencia:
    quantidades = []
    for data in datas_unicas:
        quantidades.append(datas.count(data))
    df_data = pd.DataFrame({'data':datas_unicas, 'quantidade-atendimentos':quantidades})
    dataframes_data.append(df_data)
    
#===Criação dos arquivos csv
for i in range(len(nomes_clientes)):
    dataframes_totalatendimentos[i].to_csv(f'C:\\Users\\\\downloads\\ateliedadiele\\perfilclientes\\totalatendimento\\totalatendimento-{nomes_clientes[i]}.csv', index=False)
    dataframes_mediaatendimento[i].to_csv(f'C:\\Users\\\\downloads\\ateliedadiele\\perfilclientes\\mediaatendimento\\mediaatendimento-{nomes_clientes[i]}.csv', index=False)
    dataframes_presenca[i].to_csv(f'C:\\Users\\\\downloads\\ateliedadiele\\perfilclientes\\taxapresenca\\taxapresenca-{nomes_clientes[i]}.csv', index=False)
    dataframes_data[i].to_csv(f'C:\\Users\\\\downloads\\ateliedadiele\\perfilclientes\\fluxoano\\fluxoano-{nomes_clientes[i]}.csv', index=False)

# Realizando análise geral do negócio
- Aqui é realizada a análise geral do negócio com base nas lógicas já utilizadas anteriormente

In [57]:
#===Calculando lucro total
faturamento_total = 0
faturamento_meses = []
datas = []
for i in range(len(df)):
    servicos = (df['servicos'].iloc[i]).split()
    data = df['data'].iloc[i]
    for servico in servicos:
        valor = servicos_valores[servico]
        faturamento_total += valor
        faturamento_meses.append(valor)
        datas.append(data[3:])
    
#===Calculando média mensal
media_mensal = faturamento_total / 12

df_lucros = pd.DataFrame({'Media-mensal':[int(media_mensal)], 'Faturamento-total':[faturamento_total]})
df_lucros.to_csv('C:\\Users\\josen\\downloads\\panorama_faturamento.csv', index=False)

In [60]:
#===Faturamento em cada mês
faturamento_mensal = [[] for _ in range(12)]
for i in range(len(faturamento_meses)):
    valor = faturamento_meses[i]
    data = datas[i]
    
    try:
        index = datas_unicas.index(data)
        faturamento_mensal[index].append(valor)
    except:
        pass

faturamento_mensal_final = []
for mes in faturamento_mensal:
    total = sum(mes)
    faturamento_mensal_final.append(total)

df_lucros_mensais = pd.DataFrame({'mes':datas_unicas, 'faturamento':faturamento_mensal_final})
df_lucros_mensais.to_csv('C:\\Users\\josen\\downloads\\panorama_faturamento_mensal.csv', index=False)

In [59]:
#===Lucro por tipo de serviço
tipos_de_servico = [
    'pemao',
    'mao',
    'pe',
    'spa',
    'pdp',
    'axila',
    'contorno',
    'pmao',
    'dc',
    'buco',
    'ppe',
    'sobrancelha',
    'depilacao',
    'desencravar',
    'perna',
    'blindagem',
    'nariz',
    'clareamentoax',
    'clareamento',
]

faturamento_por_servico = [[] for _ in range(19)]
for i in range(len(df)):
    servicos = df['servicos'].iloc[i]
    for servico in servicos.split():
        try:
            index = tipos_de_servico.index(servico)
            valor = servicos_valores[servico]
            faturamento_por_servico[index].append(valor)
        except:
            pass
        
faturamento_por_servico_final = []
for faturamento in faturamento_por_servico:
    faturamento_por_servico_final.append(sum(faturamento))

df_faturamento_por_servico = pd.DataFrame({'servico':tipos_de_servico, 'faturamento':faturamento_por_servico_final})
df_faturamento_por_servico.to_csv('C:\\Users\\josen\\downloads\\panorama_faturamento_por_servico.csv', index=False)

#===Total gasto por cada cliente
df_total_por_cliente = pd.DataFrame({'faturamento':total_atendimentos, 'cliente':nomes_clientes})
df_total_por_cliente.to_csv('C:\\Users\\josen\\downloads\\panorama_total_por_cliente.csv', index=False)